In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# 1. Get Stock Data From Yahoo/Naver
- 마스크 : 모나리자, 웰크론, 오공, 케이엠, 락앤락, 케이엠테크
- 필터 : 크린앤사이언스, 성창오토텍
- 대기 환경 설비: 나노, KC코트렐, KC그린홀딩스, 하츠, 코스코ICT, 한국테크놀로지, 비디아이, 누리플랜
- 공기청정기 : 대유위니아, 위닉스, 코웨이, 오텍, 솔고바이오, 신일산업, 쿠쿠홈시스, 쿠쿠홀딩스, 롯데하이마트, 에스피지, 성호전자
- 질환 관련 : 휴비츠, JW중외제약, 디에이치피코리아, 안국약품, 삼일제약, 보령제약, 조야제약, 국제약품

## Reference

[테마주 : 미세먼지 관련주 (마스크, 공기정화기, 필터, 공기청정기 관련주 외)](https://blog.naver.com/PostView.nhn?blogId=doodlee&logNo=221481539909&parentCategoryNo=&categoryNo=38&viewDate=&isShowPopularPosts=true&from=search)

[파이썬 코드 한줄로 종목코드 한번에 다운받기](https://minjejeon.github.io/learningstock/2017/09/07/download-krx-ticker-symbols-at-once.html)

## a) Get Stock Code

In [3]:
df_stock_list = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]
df_stock_list.head()
df_stock_list.to_pickle('metadata/stocklist.pkl')

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,DSR,155660,1차 비철금속 제조업,합섬섬유로프,2013-05-15,12월,홍석빈,http://www.dsr.com,부산광역시
1,GS글로벌,1250,상품 종합 도매업,"수출입업(시멘트,철강금속,전기전자,섬유,기계화학),상품중개,광업,채석업/하수처리 서...",1976-06-26,12월,김태형,http://www.gsgcorp.com,서울특별시
2,HSD엔진,82740,일반 목적용 기계 제조업,"대형선박용엔진,내연발전엔진",2011-01-04,12월,고영열,http://www.doosanengine.com,경상남도
3,KG케미칼,1390,기초 화학물질 제조업,"콘크리트혼화제, 비료, 친환경농자재, 수처리제",1989-08-25,12월,김경묵,http://www.kgchem.co.kr,울산광역시
4,LG이노텍,11070,전자부품 제조업,기타 전자부품 제조업,2008-07-24,12월,정철동,http://www.lginnotek.co.kr,서울특별시


In [4]:
df_finedust_list = pd.read_csv('metadata/finedust_stock_list.csv', index_col=False)
df_finedust_list.head()

,그룹,기업명,종목코드,상장일
0,마스크,모나리자,NaN,NaN
1,마스크,웰크론,NaN,NaN
2,마스크,오공,NaN,NaN
3,마스크,케이엠,NaN,NaN
4,마스크,락앤락,NaN,NaN


In [5]:
def get_stock_code(company_name) :
    stock_code = df_stock_list[df_stock_list['회사명'] == company_name]['종목코드']
    #print(stock_code)
    try : 
        return str(stock_code.values[0]).zfill(6)
    except Exception as e :
        print(e)
        return 0
    
def get_ipo_date(company_name) :
    ipo_date = df_stock_list[df_stock_list['회사명']==company_name]['상장일']
    
    try : 
        return pd.to_datetime(ipo_date.values[0])
    except Exception as e:
        print(e)

In [6]:
df_finedust_list['상장일'] = df_finedust_list['기업명'].apply(lambda x : get_ipo_date(x))
df_finedust_list['종목코드'] = df_finedust_list['기업명'].apply(lambda x : get_stock_code(x))
df_finedust_list.head()

index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0


,그룹,기업명,종목코드,상장일
0,마스크,모나리자,012690,1988-10-14
1,마스크,웰크론,065950,2003-07-29
2,마스크,오공,045060,2000-08-10
3,마스크,케이엠,083550,2005-12-20
4,마스크,락앤락,115390,2010-01-28


## b) Get Stock Price Data

In [7]:
from data_io import download_stock_data, load_stock_data
from data_io import download_stock_data_from_naver

In [22]:
for idx in range(0, df_finedust_list.shape[0]) :
    try : 
        print(df_finedust_list['종목코드'].iloc[idx])
        print(df_finedust_list['상장일'].iloc[idx])
        download_stock_data('data/finedust/%s.data'%df_finedust_list['종목코드'].iloc[idx],
                            df_finedust_list['종목코드'].iloc[idx],
                            int(df_finedust_list['상장일'].dt.year.iloc[idx]),
                            int(df_finedust_list['상장일'].dt.month.iloc[idx]),
                            int(df_finedust_list['상장일'].dt.day.iloc[idx]),
                            pd.Timestamp.today().year,
                            pd.Timestamp.today().month,
                            pd.Timestamp.today().day)
        print('Done')
    except Exception as e:
        print('Try to get data from Naver')
        try : 
            download_stock_data_from_naver('data/finedust/%s.data'%df_finedust_list['종목코드'].iloc[idx],
                                           df_finedust_list['종목코드'].iloc[idx],
                                           'day',
                                           '100000')
        except Exception as e : 
            print(e)       

012690
1988-10-14 00:00:00
Done
065950
2003-07-29 00:00:00
Try to get data from Naver
045060
2000-08-10 00:00:00
Try to get data from Naver
083550
2005-12-20 00:00:00
Try to get data from Naver
115390
2010-01-28 00:00:00
Done
0
NaT
Try to get data from Naver
must be str, not int
045520
2000-12-05 00:00:00
Try to get data from Naver
080470
2006-11-03 00:00:00
Try to get data from Naver
187790
2014-05-08 00:00:00
Try to get data from Naver
119650
2010-01-29 00:00:00
Done
009440
1994-11-05 00:00:00
Done
066130
2003-02-11 00:00:00
Try to get data from Naver
0
NaT
Try to get data from Naver
must be str, not int
053590
2001-08-23 00:00:00
Try to get data from Naver
148140
2017-11-09 00:00:00
Done
069140
2010-10-26 00:00:00
Try to get data from Naver
0
NaT
Try to get data from Naver
must be str, not int
044340
2000-10-24 00:00:00
Try to get data from Naver
0
NaT
Try to get data from Naver
must be str, not int
067170
2003-11-14 00:00:00
Try to get data from Naver
043100
2000-08-08 00:00:00
Try

In [9]:
load_stock_data('data/finedust/119650.data')

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2010-01-29,8350.709961,7272.310059,7272.310059,8350.709961,503263.0,6648.240723
2010-02-01,9595.019531,8668.700195,9152.599609,9124.950195,1334054.0,7264.635742
2010-02-02,10493.700195,8876.080078,9138.769531,9512.070312,702193.0,7572.832520
2010-02-03,10341.599609,9429.110352,9705.629883,9691.799805,708692.0,7715.920898
2010-02-04,9622.669922,9124.950195,9456.759766,9539.719727,309916.0,7594.846191
...,...,...,...,...,...,...
2018-12-21,4560.000000,4160.000000,4320.000000,4170.000000,345593.0,4170.000000
2018-12-24,4225.000000,4100.000000,4170.000000,4120.000000,146829.0,4120.000000
2018-12-26,4125.000000,4040.000000,4090.000000,4060.000000,114595.0,4060.000000


In [ ]:
for ticker in df_finedust_list['종목코드'] :
    try : 
        display(load_stock_data('data/finedust/%s.data'%str(ticker).zfill(6)).head())
        #display(load_stock_data('data/finedust/%s.data'%str(ticker).zfill(6)).tail())
    except Exception as e :
        print(e)

# 2. Get Finedust Data From AirKorea
- 측정기간 : 2001년 ~ 2019년 
- 측정항목 : SO2, CO, O3, NO2, PM10,PM25

## Source

[에어코리아 최종확정자료 조회](https://www.airkorea.or.kr/jfile/readDownloadFile.do?fileId=1657eebba9237&fileSeq=1)

In [23]:
pd.read_excel('externaldata/finedust/raw/2001/2001-01.xlsl')

ImportError: Missing optional dependency 'xlrd'. Install xlrd >= 1.0.0 for Excel support Use pip or conda to install xlrd.